In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack= nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1 ,28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


In [9]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flatten_image = flatten(input_image)
print(flatten_image.size())

torch.Size([3, 784])


In [13]:
layer1 = nn.Linear(in_features = 28*28, out_features=20)
hidden1 = layer1(flatten_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-2.4685e-01,  4.2901e-01,  2.6771e-02, -1.9838e-01, -6.8849e-01,
          8.8736e-01, -4.9201e-01, -1.1421e-01, -6.8779e-01, -7.7755e-02,
          8.0027e-03,  1.7341e-01,  1.3329e-01,  5.4752e-01, -4.1992e-01,
         -1.8905e-01,  1.2071e-01, -2.5851e-01,  6.3917e-01, -9.9282e-02],
        [-5.7524e-01,  3.4910e-01, -1.3640e-01, -3.3649e-01, -7.5178e-01,
          5.5776e-01, -2.9494e-01,  1.0099e-01, -5.3605e-01,  1.0750e-01,
         -2.3603e-01,  6.4627e-02,  6.5741e-01,  1.1796e-01, -2.1982e-01,
         -2.5628e-01,  1.6705e-02, -2.0111e-01,  9.3106e-01,  4.6325e-01],
        [-4.8886e-01,  1.7496e-01,  6.0016e-02, -5.2637e-01, -4.7533e-01,
          5.8058e-01,  6.7745e-02, -2.5602e-01, -8.4233e-01,  5.3498e-02,
         -5.0424e-02,  4.8903e-02,  3.3690e-01,  3.0415e-01, -2.3814e-04,
         -1.5718e-02,  3.0897e-04, -4.3673e-01,  5.5034e-01,  1.3185e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000e+00, 4.2901e-01, 2.6771e-02, 0.0

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [17]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [18]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0074,  0.0214, -0.0255,  ...,  0.0008, -0.0346, -0.0259],
        [-0.0331, -0.0333,  0.0090,  ...,  0.0118, -0.0269, -0.0139]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0296, 0.0083], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0040,  0.0390,  0.0010,  ..., -0.0005,  0.0028,  0.0326],
        [-0.0131,  0.0236, -0.0271,  ...,  0.0111, -0.0149, -0.0087]],
       device='cuda:0', grad_fn=<Slic